In [1]:
import torch
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms

# Squares

In [6]:
class SquareDataset(Dataset):
    def __init__(self, size):
        self.size = size
        self.X = torch.randint(255, (size, 9), dtype=torch.float)

        real_w = torch.tensor([[1,1,1,0,0,0,0,0,0],
                               [0,0,0,1,1,1,0,0,0],
                               [0,0,0,0,0,0,1,1,1]], 
                               dtype=torch.float)

        y = torch.argmax(self.X.mm(real_w.t()), 1)
        
        self.Y = torch.zeros(size, 3, dtype=torch.float) \
                      .scatter_(1, y.view(-1, 1), 1)

    def __getitem__(self, index):
        return (self.X[index], self.Y[index])

    def __len__(self):
        return self.size

In [10]:
squares = SquareDataset(256)
print(squares[34])
print(squares[254])
print(squares[25])

(tensor([ 11., 152., 137., 103.,  26., 183., 211., 152., 192.]), tensor([0., 0., 1.]))
(tensor([ 24., 131.,  15., 239., 169., 187.,  68.,  76.,  21.]), tensor([0., 1., 0.]))
(tensor([194.,  22.,  38., 192.,  18., 147., 104., 182.,  87.]), tensor([0., 0., 1.]))


In [17]:
dataloader = DataLoader(squares, batch_size=5)

for batch, (X, Y) in enumerate(dataloader):
    print(X, '\n\n', Y)
    break

tensor([[190., 142., 254., 201.,  17., 132., 235.,  83., 239.],
        [  6., 241., 198., 131., 149.,  11., 185., 132.,  63.],
        [ 61., 147., 254., 196., 183., 169.,  75.,  34., 123.],
        [251.,  52., 238., 109., 177., 210.,  53.,  37.,  24.],
        [ 68., 254., 216., 248., 205.,  53., 238., 190.,  63.]]) 

 tensor([[1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


# Digits

In [29]:
digits = datasets.MNIST('data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Lambda(lambda x: x.view(28*28))
                    ]),
                    target_transform=transforms.Compose([
                        transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(0, y, 1))
                    ])
                 )


In [28]:
dataloader = DataLoader(digits, batch_size=10, shuffle=True)

for batch, (X, Y) in enumerate(dataloader):
    print(X, '\n\n', Y)
    break

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]) 

 tensor([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])
